In [14]:
import polars as pl
import polars.selectors as cs
import plotly.express as px
import os
import us

parquet_data_source = 'Border_Crossing_Entry_Data.parquet'
csv_data_source = 'Border_Crossing_Entry_Data.csv' 
date_fmt ='%b-%y'

state_list = [
    'Alaska',  'Arizona', 'California', 'Idaho', 'Maine', 'Michigan',
    'Minnesota', 'Montana', 'New Mexico', 'New York', 'North Dakota', 'Texas',
    'Vermont', 'Washington'
    ]
state_abbr_list = [
    'AK', 'AZ', 'CA', 'ID', 'ME', 'MI',
    'MN', 'MT', 'NM', 'NY', 'ND', 'TX', 
    'VT', 'WA'
]


df_states = pl.DataFrame({
    'STATE' :state_list,
    'STATE_ABBR' : state_abbr_list
})


measure_list = [
    'Bus Passengers', 'Buses', 'Pedestrians', 'Personal Vehicle Passengers', 
    'Personal Vehicles', 'Rail Containers Empty', 'Rail Containers Loaded', 
    'Train Passengers', 'Trains', 'Truck Containers Empty', 
    'Truck Containers Loaded', 'Trucks'
    ]
transpo_list = [
    'Bus', 'Bus', 'Walk', 'Car', 
    'Car', 'Train', 'Train', 
    'Train', 'Train', 'Truck', 'Truck', 'Truck'
]


df_transpo = pl.DataFrame({
    'MEASURE'    : measure_list,
    'TRANSPO'    : transpo_list
})

if os.path.exists(parquet_data_source): # use pre-cleaned parquet file
    print(f'Reading data from {parquet_data_source}')
    df = (
        pl.scan_parquet(parquet_data_source)
        .with_columns(pl.col('LAT', 'LONG').cast(pl.Float64))
        .collect()
    )

else:  # read data from csv and clean
    print(f'Reading data from {csv_data_source}')
    df = (
        pl.scan_csv(csv_data_source,try_parse_dates=True)
        .rename(lambda c: c.upper()) # col names to upper case
        .select(
            PORT = pl.col('PORT NAME'),
            STATE = pl.col('STATE'),
            BORDER = pl.col('BORDER')
                .str.replace(' Border', ''),
            MEASURE = pl.col('MEASURE'),
            DATE = pl.col('DATE').str.to_date(format=date_fmt),
            VALUE = pl.col('VALUE').cast(pl.UInt32),
            POINT = pl.col('POINT')   # get rid or parens and word POINT
                .str.replace_all('POINT ', '')
                .str.strip_chars('(')
                .str.strip_chars(')')
        )
        .filter(pl.col('VALUE') > 0.0)
        .with_columns(  # LONG and LAT in point column have better resolution
            LAT = pl.col('POINT').str.split(' ').list.first().cast(pl.Float64),
            LONG = pl.col('POINT').str.split(' ').list.last().cast(pl.Float64),        
        )
        .drop('POINT')
        .drop_nulls(subset='STATE')
        .collect()
        .join(
            df_states,
            on='STATE',
            how='left'
        )
        .join(
            df_transpo,
            on='MEASURE',
            how='left'
        )
        .group_by(
            'BORDER', 'STATE', 'STATE_ABBR', 'PORT', 'TRANSPO', 
            'DATE', 'LAT', 'LONG', 
            )
            .agg(pl.col('VALUE').sum())  
    )
    port_list=sorted(df.unique('PORT').get_column('PORT').to_list())

    state_enum = pl.Enum(state_list)
    state_abbr_enum = pl.Enum(state_abbr_list)
    transpo_enum = pl.Enum(sorted(list(set(transpo_list))))
    port_enum = pl.Enum(sorted(list(set(port_list))))
    border_enum = pl.Enum(['US-Canada', 'US-Mexico'])
    df = (
        df
        .lazy()
        .with_columns(
            STATE=pl.col('STATE').cast(state_enum),
            STATE_ABBR=pl.col('STATE_ABBR').cast(state_abbr_enum),
            TRANSPO=pl.col('TRANSPO').cast(transpo_enum),
            PORT=pl.col('PORT').cast(port_enum),
            BORDER = pl.col('BORDER').cast(border_enum)
        )
        .collect()
    )



    df.write_parquet(parquet_data_source)
    # df.write_excel('df.xlsx')

print(df.glimpse())
print(df.columns)
print(df.schema)



Reading data from Border_Crossing_Entry_Data.parquet
Rows: 129023
Columns: 9
$ BORDER     <enum> US-Canada, US-Canada, US-Canada, US-Canada, US-Canada, US-Canada, US-Canada, US-Canada, US-Canada, US-Canada
$ STATE      <enum> Alaska, Alaska, Alaska, Alaska, Alaska, Alaska, Alaska, Alaska, Alaska, Alaska
$ STATE_ABBR <enum> AK, AK, AK, AK, AK, AK, AK, AK, AK, AK
$ PORT       <enum> Alcan, Alcan, Alcan, Alcan, Alcan, Alcan, Alcan, Alcan, Alcan, Alcan
$ TRANSPO    <enum> Bus, Bus, Bus, Bus, Bus, Bus, Bus, Bus, Bus, Bus
$ DATE       <date> 1996-01-01, 1996-03-01, 1996-04-01, 1996-05-01, 1996-06-01, 1996-07-01, 1996-08-01, 1996-09-01, 1996-10-01, 1996-11-01
$ LAT         <f64> -141.001444, -141.001444, -141.001444, -141.001444, -141.001444, -141.001444, -141.001444, -141.001444, -141.001444, -141.001444
$ LONG        <f64> 62.614961, 62.614961, 62.614961, 62.614961, 62.614961, 62.614961, 62.614961, 62.614961, 62.614961, 62.614961
$ VALUE       <u32> 12, 14, 23, 668, 5683, 6630, 6348, 3478, 

In [15]:
transpos = sorted(df.get_column('TRANSPO').unique().to_list())
for t in transpos:
    print(t)

Bus
Car
Train
Truck
Walk


In [16]:
df_transpo = (
    df
    .unique('MEASURE')
    .select('MEASURE', 'TRANSPO')
    .sort('MEASURE')
)
print(df_transpo.get_column('MEASURE').to_list())
print(df_transpo.get_column('TRANSPO').to_list())


ColumnNotFoundError: "MEASURE" not found

In [ ]:
import us
state_list = sorted(df.unique('STATE').get_column('STATE').to_list())
state_list


In [ ]:
for col in df.columns:
    print(df[col].value_counts().sort('count'))

In [ ]:
df.sample(10).glimpse()

#### select, rename, cast columns

In [ ]:
for i, col in enumerate(df.columns):
    select_string = (
        f"{col.upper().replace(' ', '_')} " + 
        f"= pl.col('{col}'), # .cast(pl.String)',"
    )

    print(select_string)
    # print(f"{col.upper().replace(' ', '_')} = pl.col('{col} # .cast(pl.String)'),")

In [ ]:
state_list = ['Alaska',  'Arizona', 'California', 'Idaho', 'Maine', 'Michigan',
    'Minnesota', 'Montana', 'New Mexico', 'New York', 'North Dakota', 'Texas',
    'Vermont', 'Washington']
state_abbr_list = []
for state in state_list:
    state_abbr_list.append(us.states.lookup(state).abbr)
print(state_abbr_list)

In [ ]:
df.unique(['TRANSPO', 'MEASURE'])

In [20]:
fig = px.choropleth(
    df.group_by('STATE', 'STATE_ABBR').agg(pl.col('VALUE').sum()),
    locations='STATE_ABBR', 
    locationmode='USA-states', 
    scope='usa',
    color='VALUE',
    template='plotly_white',
    color_continuous_scale="Viridis_r", 
)
fig.show()

In [25]:
selected_state = 'New York'
df_state = (
    df
    .filter(pl.col('STATE') == selected_state)
    .group_by('STATE', 'STATE_ABBR', 'PORT', 'LAT', 'LONG')
    .agg(pl.col('VALUE').sum())
    .sort('PORT')
)
print('df_state')
print(df_state)

# Create a scatter map

fig_state_port_map = px.scatter_map(
    df_state,
    lon='LAT',
    lat='LONG',
    # hover_name='PORT',
    text='PORT',
    size='VALUE',
    size_max=15,
    zoom=5,
    # title=f'Border Crossing in {selected_state}',
    # map_style='carto-positron'  # Use MapLibre-compatible tile style
)
fig_state_port_map.write_html('map.html')
fig_state_port_map.show()

df_state
shape: (7, 6)
┌──────────┬────────────┬────────────────────────┬────────────┬───────────┬───────────┐
│ STATE    ┆ STATE_ABBR ┆ PORT                   ┆ LAT        ┆ LONG      ┆ VALUE     │
│ ---      ┆ ---        ┆ ---                    ┆ ---        ┆ ---       ┆ ---       │
│ enum     ┆ enum       ┆ enum                   ┆ f64        ┆ f64       ┆ u32       │
╞══════════╪════════════╪════════════════════════╪════════════╪═══════════╪═══════════╡
│ New York ┆ NY         ┆ Alexandria Bay         ┆ -75.983592 ┆ 44.347229 ┆ 72103259  │
│ New York ┆ NY         ┆ Buffalo Niagara Falls  ┆ -79.055847 ┆ 43.09549  ┆ 614735269 │
│ New York ┆ NY         ┆ Cape Vincent           ┆ -76.352489 ┆ 44.134917 ┆ 365490    │
│ New York ┆ NY         ┆ Champlain Rouses Point ┆ -73.452663 ┆ 45.008833 ┆ 129754120 │
│ New York ┆ NY         ┆ Massena                ┆ -74.739722 ┆ 44.990556 ┆ 85984654  │
│ New York ┆ NY         ┆ Ogdensburg             ┆ -75.45775  ┆ 44.73309  ┆ 27048025  │
│ New Yor